#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
* https://kidger.site/thoughts/jaxtyping/
* https://github.com/patrick-kidger/torchtyping/tree/master

## Задачи для совместного разбора

In [1]:
# !pip install torchtyping

In [2]:
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked
import torch as th

Scalar = TensorType[()]
patch_typeguard()

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [3]:
class Neuron:
    def __init__(self, n_features: int, bias: float) -> None:
        self.weights: TensorType["n_features"] = th.rand(n_features)
        self.bias: float = bias

    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> Scalar:
        return inputs @ self.weights + self.bias

In [4]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

In [5]:
neuron = Neuron(n_features=4, bias=0.0)
neuron.forward(inputs)

tensor(6.7962)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [6]:
class ReLU:
    @typechecked
    def forward(self, inputs: TensorType["n_features"]) -> TensorType["n_features"]:
        return th.clip(inputs, min=0)

In [7]:
inputs = th.tensor([1.0, -2.0, 3.0, -4.0])

act = ReLU()
act.forward(inputs)

tensor([1., 0., 3., 0.])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [8]:
class MSELoss:
    @typechecked
    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"]
    ) -> Scalar:
        return ((y_pred - y_true)**2).mean()

In [9]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

In [10]:
criterion = MSELoss()
loss = criterion.forward(y_pred, y_true)

In [11]:
loss

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [x] Проверено на семинаре

In [12]:
from typing import Any

In [13]:
class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.n_neurons = n_neurons
        self.n_features = n_features

        fgen = th.randn if bias else th.zeros
        self.weights: TensorType["n_neurons", "n_features"] = th.randn((n_neurons, n_features))
        self.bias_: TensorType["n_neurons"] = fgen((self.n_neurons,))
    
    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "n_neurons"]:
        return inputs @ self.weights.T + self.bias_
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [14]:
linear = Linear(2, 4, bias=False)
linear(th.randn((3, 4)))

tensor([[-0.6151, -0.1395],
        [ 0.0430, -0.8772],
        [-0.3652,  1.2548]])

In [15]:
linear = Linear(2, 4, bias=True)
linear(th.randn((3, 4)))

tensor([[ 0.2063,  1.8482],
        [ 0.5031,  1.8048],
        [-0.5639,  2.2618]])

<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

- [x] Проверено на семинаре

In [16]:
linear1 = Linear(10, 4, bias=False)
linear2 = Linear(7, linear1.n_neurons, bias=False)

In [17]:
out1 = linear1(th.randn((3, 4)))
out2 = linear2(out1)
out2

tensor([[-10.0178,   9.3341,  -9.0969,   1.5847,  -7.6655,  -4.5518,  -7.2286],
        [ -8.1931,  -0.0842,  10.1554,  27.8915,  16.4471,  10.2863,   1.7852],
        [ -4.2112,   5.4299,  -8.3019,  -6.7700,  -5.3522,  -3.4097,  -2.3064]])

In [18]:
out1.shape

torch.Size([3, 10])

In [19]:
out2.shape

torch.Size([3, 7])

### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

- [x] Проверено на семинаре

In [20]:
class Softmax:
    def __init__(self) -> None:
        pass

    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        exp: TensorType["batch", "feats"] = th.exp(inputs)
        return exp / exp.sum(dim=1, keepdim=True)
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [21]:
inputs = th.randn((4, 3))
inputs

tensor([[-0.8381,  0.3734,  1.0092],
        [ 0.0616,  0.4078, -0.4896],
        [-0.2737,  2.0174,  0.5014],
        [ 1.5901,  0.0868, -0.1410]])

In [22]:
softmax = Softmax()
softmax(inputs)

tensor([[0.0934, 0.3139, 0.5927],
        [0.3345, 0.4728, 0.1927],
        [0.0766, 0.7571, 0.1663],
        [0.7146, 0.1589, 0.1265]])

<p class="task" id="4"></p>

4 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

- [x] Проверено на семинаре

In [23]:
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha

    @typechecked()
    def forward(self, inputs: TensorType["batch", "feats"]) -> TensorType["batch", "feats"]:
        return th.where(inputs >= 0, inputs, self.alpha*(th.exp(inputs) - 1))
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [24]:
inputs = th.randn((4, 3))
inputs

tensor([[ 0.3564, -0.3909, -0.6493],
        [ 0.1749,  2.0067,  0.1827],
        [ 0.7416,  0.9053, -0.0574],
        [ 1.0837, -0.9181,  0.5992]])

In [25]:
elu = ELU(0.001)
elu(inputs)

tensor([[ 3.5638e-01, -3.2355e-04, -4.7757e-04],
        [ 1.7487e-01,  2.0067e+00,  1.8269e-01],
        [ 7.4160e-01,  9.0533e-01, -5.5826e-05],
        [ 1.0837e+00, -6.0071e-04,  5.9917e-01]])

### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

- [x] Проверено на семинаре

In [26]:
class CrossEntropyLoss:
    def __init__(self) -> None:
        pass

    @typechecked()
    def forward(self, y_pred: TensorType["batch", "classes", float], y_true: TensorType["batch", int]):
        return -(th.log(y_pred).gather(1, y_true.unsqueeze(1))).mean()
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [34]:
layer = Linear(2, 4, bias=True)
out = layer.forward(th.randn(10, 4))
out

tensor([[-0.6662,  0.0296,  2.2194,  ...,  0.6275, -1.8581,  2.2937],
        [-0.7578, -0.1378,  1.5261,  ...,  1.4549, -1.6439,  2.6303],
        [ 3.8375, -6.1670,  3.4797,  ...,  2.1404,  0.5485,  0.3206],
        ...,
        [ 0.9514, -1.5089,  1.6102,  ...,  2.0724,  0.0574,  0.9945],
        [-1.8693,  1.5886,  0.4920,  ..., -0.2532, -1.6078,  2.1432],
        [ 1.4529, -1.1024, -0.4982,  ...,  4.9088,  2.3361,  0.0996]])

In [28]:
softmax = Softmax()
y_pred = softmax.forward(out)
y_pred

tensor([[0.9673, 0.0327],
        [0.7394, 0.2606],
        [0.8000, 0.2000],
        [0.0389, 0.9611],
        [0.7691, 0.2309],
        [0.6398, 0.3602],
        [0.0542, 0.9458],
        [0.6290, 0.3710],
        [0.1661, 0.8339],
        [0.6045, 0.3955]])

In [29]:
loss = CrossEntropyLoss()
loss.forward(y_pred, y_true=th.randint(0, 2, (10,)))

tensor(1.0427)

<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [x] Проверено на семинаре

In [30]:
from typing import Any


class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    @typechecked
    def data_loss(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
    ) -> Scalar:
        return ((y_true - y_pred) ** 2).mean()

    @typechecked
    def reg_loss(self, weights: TensorType["batch", 1]) -> Scalar:
        return th.sum(weights ** 2)

    @typechecked()
    def forward(
        self,
        y_pred: TensorType["batch"],
        y_true: TensorType["batch"],
        weights: TensorType["batch", 1],
    ) -> Scalar:
        return self.data_loss(y_pred, y_true) + self.lambda_ * self.reg_loss(weights)

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        return self.forward(*args, **kwds)

In [31]:
y_pred = th.randn(2)
y_true = th.randn(2)
weights = th.randn(2).unsqueeze(1)
y_true, y_pred, th.sum(weights**2)

(tensor([0.3423, 1.3200]), tensor([ 0.7011, -1.1421]), tensor(1.3572))

In [32]:
loss_fn = MSERegularized(lambda_=0.001)
loss_fn(y_pred, y_true, weights)

tensor(3.0968)